# Navie Bayes: 스팸문자 분류



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


np.random.seed(2021)

In [2]:
#colab에서 .csv 파일 가져오기
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab/study/dataset

import pandas as pd
spam = pd.read_csv("sms_spam.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab/study/dataset


In [3]:
text = spam["text"]
label = spam["type"]

In [4]:
# data EDA

print(text[0])
print(label[0])


Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham


In [5]:
label.value_counts()

ham     4827
spam     747
Name: type, dtype: int64

In [6]:
# data Cleaning
# 정답의 문자를 숫자로 변환
# ham은 0으로, spam은 1로 변환

label = label.map({"ham": 0, "spam": 1})
label.value_counts()

0    4827
1     747
Name: type, dtype: int64

In [7]:
# text를 문자만 존재하도록 정리
# regex를 통해 영어, 숫자 그리고 띄어쓰기를 제외한 모든 단어를 지움

re_pattern = "[^a-zA-Z0-9\ ]"

In [8]:
text.iloc[:1].str.replace(re_pattern, "", regex=True)[0]

'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [9]:
text = text.str.replace(re_pattern, "", regex=True)

In [10]:
# 대문자들을 모두 소문자로 바꿔줌
text.iloc[:1].str.lower()[0]

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

In [11]:
text = text.str.lower()

In [12]:
# data aplit
from sklearn.model_selection import train_test_split

train_text, test_text, train_label, test_label = train_test_split(
    text, label, train_size=0.7, random_state=2021
)

print(f"train_data size: {len(train_label)}, {len(train_label)/len(text):.2f}")
print(f"test_data size: {len(test_label)}, {len(test_label)/len(text):.2f}")

train_data size: 3901, 0.70
test_data size: 1673, 0.30


In [13]:
# count vector
# word tokenize
# 문장을 단어로 나누는 데에는 `nltk` 패키지의 `word_tokenize`를 이용

import nltk
from nltk import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
train_text.iloc[0]

'am only searching for good dual sim mobile pa'

In [15]:
word_tokenize(train_text.iloc[0])

['am', 'only', 'searching', 'for', 'good', 'dual', 'sim', 'mobile', 'pa']

In [16]:
# count vectorize
from sklearn.feature_extraction.text import CountVectorizer

train_text.iloc[:2].values

array(['am only searching for good dual sim mobile pa',
       'excellent ill see what rileys plans are'], dtype=object)

In [17]:
cnt_vectorizer = CountVectorizer(tokenizer=word_tokenize)
cnt_vectorizer.fit(train_text.iloc[:2])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(tokenizer=<function word_tokenize at 0x7f3d53b88160>)

In [18]:
# 문장에서 나온 단어
cnt_vectorizer.vocabulary_

{'am': 0,
 'only': 8,
 'searching': 12,
 'for': 4,
 'good': 5,
 'dual': 2,
 'sim': 14,
 'mobile': 7,
 'pa': 9,
 'excellent': 3,
 'ill': 6,
 'see': 13,
 'what': 15,
 'rileys': 11,
 'plans': 10,
 'are': 1}

In [19]:
vocab = sorted(cnt_vectorizer.vocabulary_.items(), key=lambda x: x[1])
vocab = list(map(lambda x: x[0], vocab))
vocab

['am',
 'are',
 'dual',
 'excellent',
 'for',
 'good',
 'ill',
 'mobile',
 'only',
 'pa',
 'plans',
 'rileys',
 'searching',
 'see',
 'sim',
 'what']

In [20]:
sample_cnt_vector = cnt_vectorizer.transform(train_text.iloc[:2]).toarray()
sample_cnt_vector

array([[1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1]])

In [21]:
train_text.iloc[:2].values

array(['am only searching for good dual sim mobile pa',
       'excellent ill see what rileys plans are'], dtype=object)

In [22]:
pd.DataFrame(sample_cnt_vector, columns=vocab)

,am,are,dual,excellent,for,good,ill,mobile,only,pa,plans,rileys,searching,see,sim,what
0,1,0,1,0,1,1,0,1,1,1,0,0,1,0,1,0
1,0,1,0,1,0,0,1,0,0,0,1,1,0,1,0,1


In [23]:
# 학습
cnt_vectorizer = CountVectorizer(tokenizer=word_tokenize)
cnt_vectorizer.fit(train_text)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(tokenizer=<function word_tokenize at 0x7f3d53b88160>)

In [24]:
#전체 단어 개수
len(cnt_vectorizer.vocabulary_)

7908

In [25]:
#예측
train_matrix = cnt_vectorizer.transform(train_text)
test_matrix = cnt_vectorizer.transform(test_text)

In [26]:

cnt_vectorizer.transform(["notavailblewordforcnt"]).toarray().sum()

0

In [27]:
# Naive Bayes
from sklearn.naive_bayes import BernoulliNB

naive_bayes = BernoulliNB()

In [28]:
# 학습
naive_bayes.fit(train_matrix, train_label)

BernoulliNB()

In [29]:
# 예측
train_pred = naive_bayes.predict(train_matrix)
test_pred = naive_bayes.predict(test_matrix)

In [30]:
# 평가
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(train_label, train_pred)
test_acc = accuracy_score(test_label, test_pred)

print(f"Train Accuracy is {train_acc:.4f}")
print(f"Test Accuracy is {test_acc:.4f}")

Train Accuracy is 0.9854
Test Accuracy is 0.9767
